In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
global details

In [2]:
details=[]

In [4]:
#get bowling type
def get_bowlingtype(string):
    hand="Right"
    type="Spin"
    if "right" in string.lower():
        hand= "Right"
            
    elif "left" in string.lower():
        hand= "Left"
        
    if "medium" in string.lower() or "fast" in string.lower():
        type="Fast"
    if "spin" in string.lower() or "off" in string.lower() or "leg" in string.lower() or "chinaman" in string.lower() or "break" in string.lower():
        type="Spin"
    
    final=''
    
    if hand =="Right":
        final= "வலதுகை" +" "
    elif hand =="Left":
        final = "இடதுகை" +" "
    
    if type== "Fast":
        final+="வேக பந்துவீச்சு"
    elif type == "Spin":
        final+="சுழல் பந்துவீச்சு"
            
    return str(final)

In [5]:
#get batting type
def get_battingtype(string):
    hand=""

    if "right" in string.lower():
        hand= "வலதுகை துடுப்பாட்டம்"
            
    else:
        hand= "இடதுகை துடுப்பாட்டம்"
        
    return str(hand)

In [6]:
def get_teamdetail(string):
    final_team=''
    team = ["New Zealand","India","Australia","England","Pakistan", "South Africa", "West Indies","Sri Lanka","Bangladesh","Zimbabwe","Afghanistan"]
    team_tran= ["நியூசிலாந்து","இந்தியா","அஸ்திரேலியா","இங்கிலாந்து", "பாகிஸ்தான்", "தென்னாப்பிரிக்கா", "மேற்கிந்திய தீவுகள்", "இலங்கை", "வங்கதேசம்", "சிம்பாப்வே", "ஆப்கானிஸ்தான்"]
    for j in range(len(team)):
        if team[j][0:2]==string[0:2]:
            final_team=team_tran[j]
            break
    return final_team

In [7]:
def get_playerRole(string):
    if string=="Batsman":
        return "துடுப்பாட்ட வீரர்"
    elif string =="Bowler":
        return "பந்து வீச்சாளர்"
    elif "WK" in string:
        return "காப்பாளர்"
    else: 
        return "சகலதுறை வீரர்"
    

In [36]:

def getdetails(req,req_des):
    soup= BeautifulSoup(req.content,"html.parser")
    name=str(soup.find_all('h1',class_="cb-font-40")[0])
    tags=soup.find_all('div',class_= "cb-col cb-col-60 cb-lst-itm-sm")
    numtags=soup.find_all('td',class_= "text-right")

    age=str(tags[0])
    role=str(tags[2])
    batStyle=str(tags[3])
    bowlStyle=str(tags[4])
    teams=str(tags[-1])
    runs=str(numtags[16])
    wickets=str(numtags[-20])

    result = re.search('>(.*)<', runs)
    playerOdiRuns=result.group(1)
    result = re.search('>(.*)<', wickets)
    playerOdiWickets=result.group(1)

    result = re.search('>(.*)<', name)
    playerName=result.group(1)
    
    playerAge=int(age[age.find("years")-3:age.find("years")-1])
        
    result = re.search('>(.*)<', role)
    playerRole=result.group(1)[1:-1]
    result = re.search('>(.*)<', batStyle)
    playerBatStyle=result.group(1)[1:-1]
    
    result = re.search('>(.*)<', bowlStyle)
    playerBowlStyle=result.group(1)[1:-1]
    
    result = re.search('>(.*)<', teams)
    playerTeams=result.group(1)[1:-1].split(",")
    
    
    soup= BeautifulSoup(req_des.content,"html.parser")
    destag=soup.find_all('p')

    mainBio=''
    count=0
    for j in range(len(destag)):
        if len(str(destag[j]))> 150 :
            count+=1
            mainBio=mainBio+str(destag[j])
            break

    mainBio=mainBio.replace("<p>","")
    mainBio=mainBio.replace("</p>","")
    mainBio=mainBio.replace("(","")
    mainBio=mainBio.replace(")","")

    mainBio=mainBio.split(".")
    bioPiece=[]
    for j in mainBio:
        bioPiece.extend(j.split(","))

    cleanedBioPiece=[]
    for p in bioPiece:
        w=p.replace('and','')
        if len(w)>0:
            if w[0]==" ":
                w=w[1:]
            if len(w)>0:
                if w[0]==" ":
                    w=w[1:]

        if len(w)>20:
            cleanedBioPiece.append(w)
    
    
    if playerOdiWickets == '-':
        playerOdiWickets=str("0")
        
    corpus = []
    corpus.append({
        "பெயர்": playerName,
        "வயது": playerAge,
        "வகை": get_playerRole(playerRole),
        "துடுப்பாட்டம்": get_battingtype(playerBatStyle),
        "பந்துவீச்சு": get_bowlingtype(playerBowlStyle),
        "அணி": get_teamdetail(playerTeams[0]),
        "இலக்குகள்": playerOdiWickets,
        "ஓட்டங்கள்": playerOdiRuns,
        
    })

    df2=[playerName,playerAge,get_playerRole(playerRole),get_battingtype(playerBatStyle),get_bowlingtype(playerBowlStyle),get_teamdetail(playerTeams[0]),playerOdiWickets,playerOdiRuns]
    details.append(df2)

    import json
    with open(str(playerName)+'.json', 'w') as f:
        json.dump(corpus, f)
        
    
    print(corpus)


In [37]:
with open("players_url.txt") as f:
    contents = f.readlines()
    
with open("players_des_url.txt") as f:
    players_des_contents = f.readlines()



In [38]:

for j in range(len(contents)):
    req = requests.get(contents[j])
    req_des=requests.get(players_des_contents[j])
    getdetails(req,req_des)

[{'பெயர்': 'Virat Kohli', 'வயது': 33, 'வகை': 'துடுப்பாட்ட வீரர்', 'துடுப்பாட்டம்': 'வலதுகை துடுப்பாட்டம்', 'பந்துவீச்சு': 'வலதுகை வேக பந்துவீச்சு', 'அணி': 'இந்தியா', 'இலக்குகள்': '4', 'ஓட்டங்கள்': '12169'}]
[{'பெயர்': 'Rohit Sharma', 'வயது': 34, 'வகை': 'துடுப்பாட்ட வீரர்', 'துடுப்பாட்டம்': 'வலதுகை துடுப்பாட்டம்', 'பந்துவீச்சு': 'வலதுகை சுழல் பந்துவீச்சு', 'அணி': 'இந்தியா', 'இலக்குகள்': '1', 'ஓட்டங்கள்': '9205'}]
[{'பெயர்': 'Ross Taylor', 'வயது': 37, 'வகை': 'துடுப்பாட்ட வீரர்', 'துடுப்பாட்டம்': 'வலதுகை துடுப்பாட்டம்', 'பந்துவீச்சு': 'வலதுகை சுழல் பந்துவீச்சு', 'அணி': 'நியூசிலாந்து', 'இலக்குகள்': '0', 'ஓட்டங்கள்': '8576'}]
[{'பெயர்': 'Jonny Bairstow', 'வயது': 32, 'வகை': 'காப்பாளர்', 'துடுப்பாட்டம்': 'வலதுகை துடுப்பாட்டம்', 'பந்துவீச்சு': 'வலதுகை வேக பந்துவீச்சு', 'அணி': 'இங்கிலாந்து', 'இலக்குகள்': '0', 'ஓட்டங்கள்': '3498'}]
[{'பெயர்': 'David Warner', 'வயது': 35, 'வகை': 'துடுப்பாட்ட வீரர்', 'துடுப்பாட்டம்': 'இடதுகை துடுப்பாட்டம்', 'பந்துவீச்சு': 'வலதுகை சுழல் பந்துவீச்சு', 'அணி': 'அஸ்திரே

In [39]:
import pandas as pd
columns=["பெயர்","வயது", "வகை","துடுப்பாட்டம்","பந்துவீச்சு","அணி","இலக்குகள்","ஓட்டங்கள்"]
df = pd.DataFrame(details, columns =columns)

In [43]:
df

,பெயர்,வயது,வகை,துடுப்பாட்டம்,பந்துவீச்சு,அணி,இலக்குகள்,ஓட்டங்கள்
0,Virat Kohli,33,துடுப்பாட்ட வீரர்,வலதுகை துடுப்பாட்டம்,வலதுகை வேக பந்துவீச்சு,இந்தியா,4,12169
1,Rohit Sharma,34,துடுப்பாட்ட வீரர்,வலதுகை துடுப்பாட்டம்,வலதுகை சுழல் பந்துவீச்சு,இந்தியா,1,9205
2,Ross Taylor,37,துடுப்பாட்ட வீரர்,வலதுகை துடுப்பாட்டம்,வலதுகை சுழல் பந்துவீச்சு,நியூசிலாந்து,0,8576
3,Jonny Bairstow,32,காப்பாளர்,வலதுகை துடுப்பாட்டம்,வலதுகை வேக பந்துவீச்சு,இங்கிலாந்து,0,3498
4,David Warner,35,துடுப்பாட்ட வீரர்,இடதுகை துடுப்பாட்டம்,வலதுகை சுழல் பந்துவீச்சு,அஸ்திரேலியா,0,5455
...,...,...,...,...,...,...,...,...
301,Kevin Pietersen,41,துடுப்பாட்ட வீரர்,வலதுகை துடுப்பாட்டம்,வலதுகை சுழல் பந்துவீச்சு,இங்கிலாந்து,1,4440
302,Shane Watson,40,சகலதுறை வீரர்,வலதுகை துடுப்பாட்டம்,வலதுகை வேக பந்துவீச்சு,அஸ்திரேலியா,48,5757
303,Brendon McCullum,40,காப்பாளர்,வலதுகை துடுப்பாட்டம்,வலதுகை சுழல் பந்துவீச்சு,நியூசிலாந்து,0,6083
304,Scott Styris,46,சகலதுறை வீரர்,வலதுகை துடுப்பாட்டம்,வலதுகை வேக பந்துவீச்சு,நியூசிலாந்து,18,4483


In [18]:
df["பந்துவீச்சு"].describe()

count                          200
unique                           4
top       வலதுகை சுழல் பந்துவீச்சு
freq                            92
Name: பந்துவீச்சு, dtype: object

In [19]:
df.describe()

,வயது
count,200.000000
mean,33.560000
std,6.839613
min,20.000000
25%,29.750000
50%,33.000000
75%,37.000000
max,52.000000


In [40]:
df.to_csv("playerDetails.csv")

In [44]:
df.to_json("sample.json")

In [41]:
import csv 
import json 

def csv_to_json(csvFilePath, jsonFilePath):
    jsonArray = []
      
    #read csv file
    with open(csvFilePath, encoding='utf-8') as csvf: 
        #load csv file data using csv library's dictionary reader
        csvReader = csv.DictReader(csvf) 

        #convert each csv row into python dict
        for row in csvReader: 
            #add this python dict to json array
            jsonArray.append(row)
  
    #convert python jsonArray to JSON String and write to file
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf: 
        jsonString = json.dumps(jsonArray, indent=4)
        jsonf.write(jsonString)
          
csvFilePath = r'playerDetails.csv'
jsonFilePath = r'playerDetailsJson.json'
csv_to_json(csvFilePath, jsonFilePath)


In [42]:
with open('playerDetailsJson.json', 'r') as j:
    json_data = json.load(j)

for j in json_data:
    print(j)
    print()

{'': '0', 'பெயர்': 'Virat Kohli', 'வயது': '33', 'வகை': 'துடுப்பாட்ட வீரர்', 'துடுப்பாட்டம்': 'வலதுகை துடுப்பாட்டம்', 'பந்துவீச்சு': 'வலதுகை வேக பந்துவீச்சு', 'அணி': 'இந்தியா', 'இலக்குகள்': '4', 'ஓட்டங்கள்': '12169'}

{'': '1', 'பெயர்': 'Rohit Sharma', 'வயது': '34', 'வகை': 'துடுப்பாட்ட வீரர்', 'துடுப்பாட்டம்': 'வலதுகை துடுப்பாட்டம்', 'பந்துவீச்சு': 'வலதுகை சுழல் பந்துவீச்சு', 'அணி': 'இந்தியா', 'இலக்குகள்': '1', 'ஓட்டங்கள்': '9205'}

{'': '2', 'பெயர்': 'Ross Taylor', 'வயது': '37', 'வகை': 'துடுப்பாட்ட வீரர்', 'துடுப்பாட்டம்': 'வலதுகை துடுப்பாட்டம்', 'பந்துவீச்சு': 'வலதுகை சுழல் பந்துவீச்சு', 'அணி': 'நியூசிலாந்து', 'இலக்குகள்': '0', 'ஓட்டங்கள்': '8576'}

{'': '3', 'பெயர்': 'Jonny Bairstow', 'வயது': '32', 'வகை': 'காப்பாளர்', 'துடுப்பாட்டம்': 'வலதுகை துடுப்பாட்டம்', 'பந்துவீச்சு': 'வலதுகை வேக பந்துவீச்சு', 'அணி': 'இங்கிலாந்து', 'இலக்குகள்': '0', 'ஓட்டங்கள்': '3498'}

{'': '4', 'பெயர்': 'David Warner', 'வயது': '35', 'வகை': 'துடுப்பாட்ட வீரர்', 'துடுப்பாட்டம்': 'இடதுகை துடுப்பாட்டம்', 'பந்துவீ